In [ ]:
!pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-hqizmfqn
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-hqizmfqn
  Resolved https://github.com/huggingface/transformers to commit a5c061d24e31706c18670c0ae6579b5c72d545f3
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.2/516.2 kB 15.9 MB/s eta 0:00:00
  Created wheel for transformers: filename=transformers-5.0.0.dev0-py3-none-any.whl size=10861879 sha256=34d047ab2ec9569c442dc8a7c09e6cee23a22595a1b87130cecce6e1c201e2fe
  Stored in directory: /tmp/pip-ephem-wheel-cache-kkldqq0z/wheels/49/a7/50/c9fdabbf10e51bb1256adb0c1a587fedd7184f5bad28d47fe3
Successfully built transformers
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.36.0
    Uninstalling huggingface-hub-0.36.0:
 

In [ ]:
from transformers import Sam3Processor, Sam3Model
import torch
from PIL import Image
import requests

device = "cuda" if torch.cuda.is_available() else "cpu"

model = Sam3Model.from_pretrained("facebook/sam3").to(device)
processor = Sam3Processor.from_pretrained("facebook/sam3")

config.json:   0%|          | 0.00/25.8k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

Loading weights:   0%|          | 0/1468 [00:00<?, ?it/s]

processor_config.json:   0%|          | 0.00/1.71k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/3.64M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/588 [00:00<?, ?B/s]

In [ ]:
!pip install opencv-python

In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!cp /content/drive/MyDrive/CV\ Final-Proj/Tractatus/tractatus.zip .

In [ ]:
!unzip tractatus.zip

Archive:  tractatus.zip
   creating: tractatus/
  inflating: tractatus/img9.jpg      
  inflating: __MACOSX/tractatus/._img9.jpg  
  inflating: tractatus/img8.jpg      
  inflating: __MACOSX/tractatus/._img8.jpg  
  inflating: tractatus/img5.jpg      
  inflating: __MACOSX/tractatus/._img5.jpg  
  inflating: tractatus/img4.jpg      
  inflating: __MACOSX/tractatus/._img4.jpg  
  inflating: tractatus/img6.jpg      
  inflating: __MACOSX/tractatus/._img6.jpg  
  inflating: tractatus/img7.jpg      
  inflating: __MACOSX/tractatus/._img7.jpg  
  inflating: tractatus/img3.jpg      
  inflating: __MACOSX/tractatus/._img3.jpg  
  inflating: tractatus/img2.jpg      
  inflating: __MACOSX/tractatus/._img2.jpg  
  inflating: tractatus/img1.jpg      
  inflating: __MACOSX/tractatus/._img1.jpg  
  inflating: tractatus/img16.jpg     
  inflating: __MACOSX/tractatus/._img16.jpg  
  inflating: tractatus/img14.jpg     
  inflating: __MACOSX/tractatus/._img14.jpg  
  inflating: tractatus/img15.jpg     

In [ ]:
!mkdir tractatus_plant_images_masks

In [ ]:
import os
import cv2
from tqdm import tqdm
import pickle as pkl
manuscript = "tractatus"
plant_files = os.listdir(f"{manuscript}_plant_images")
prompt="plant"
scores = []
if not os.path.isdir(f"{manuscript}_plant_images_{prompt}_masks"):
  os.mkdir(f"{manuscript}_plant_images_{prompt}_masks")

for pf in tqdm(plant_files):
  pf_full = f"{manuscript}_plant_images/" + pf
  image = Image.open(pf_full)

  # Segment using text prompt
  inputs = processor(images=image, text=prompt, return_tensors="pt").to(device)

  with torch.no_grad():
      outputs = model(**inputs)

  # Post-process results
  results = processor.post_process_instance_segmentation(
      outputs,
      threshold=0.5,
      mask_threshold=0.5,
      target_sizes=inputs.get("original_sizes").tolist()
  )[0]
  cv_im = cv2.imread(pf_full, cv2.IMREAD_GRAYSCALE)
  imgcopy = cv_im.copy()
  for i, mask in enumerate(results['masks']):
    cpu_mask = mask.cpu()
    imgcopy[cpu_mask == 1] = 255
    imgcopy[cpu_mask == 0] = 0
    img_name_minus_ending = pf.split(".")[0]
    confidence_score = results['scores'][i]
    scores.append((f"{img_name_minus_ending}_{i+1}", confidence_score))
    cv2.imwrite(f"{manuscript}_plant_images_{prompt}_masks/{img_name_minus_ending}_{i + 1}.jpeg", imgcopy)
with open(f"{manuscript}_plant_images_{prompt}_masks_scores.txt", "wb") as f:
  pkl.dump(scores, f)

100%|██████████| 16/16 [00:31<00:00,  1.99s/it]


In [ ]:
!zip -r plant_images_flower_masks.zip plant_images_flower_masks/

  adding: plant_images_flower_masks/ (stored 0%)
  adding: plant_images_flower_masks/page119_34.jpeg (deflated 94%)
  adding: plant_images_flower_masks/page53_4.jpeg (deflated 95%)
  adding: plant_images_flower_masks/page54_1.jpeg (deflated 91%)
  adding: plant_images_flower_masks/page103_8.jpeg (deflated 96%)
  adding: plant_images_flower_masks/page76_3.jpeg (deflated 94%)
  adding: plant_images_flower_masks/page53_15.jpeg (deflated 94%)
  adding: plant_images_flower_masks/page46_3.jpeg (deflated 90%)
  adding: plant_images_flower_masks/page40_4.jpeg (deflated 97%)
  adding: plant_images_flower_masks/page29_1.jpeg (deflated 93%)
  adding: plant_images_flower_masks/page72_2.jpeg (deflated 96%)
  adding: plant_images_flower_masks/page40_33.jpeg (deflated 96%)
  adding: plant_images_flower_masks/page15_3.jpeg (deflated 90%)
  adding: plant_images_flower_masks/page59_1.jpeg (deflated 91%)
  adding: plant_images_flower_masks/page180_6.jpeg (deflated 89%)
  adding: plant_images_flower_masks

In [ ]:
import os
import cv2
from tqdm import tqdm
import matplotlib.pyplot as plt
manuscript = "tractatus"
prompt = "leaf"
if not os.path.isdir(f"{manuscript}_plant_images_{prompt}_cleaned"):
  os.mkdir(f"{manuscript}_plant_images_{prompt}_cleaned")

mask_file = f"{manuscript}_plant_images_{prompt}_masks"
plant_files = os.listdir(mask_file)
og_img_dir = f"{manuscript}_plant_images"
for pf in plant_files:
  og_img_name = pf.split("_")[0] + ".jpg"
  print(og_img_name)
  print(f"{og_img_dir}/{og_img_name}")
  og_img = cv2.imread(f"{og_img_dir}/{og_img_name}", cv2.IMREAD_COLOR)
  og_img = og_img.copy()
  mask = cv2.imread(f"{mask_file}/{pf}", cv2.IMREAD_GRAYSCALE)
  og_img[:,:,0][mask == 0] = 0
  og_img[:,:,1][mask == 0] = 0
  og_img[:,:,2][mask == 0] = 0
  img_name_minus_ending = pf[:-5]
  cv2.imwrite(f"{manuscript}_plant_images_{prompt}_cleaned/{img_name_minus_ending}_cleaned.jpeg", og_img)
  # plt.imshow(og_img)
  # plt.show()


img16.jpg
tractatus_plant_images/img16.jpg
img12.jpg
tractatus_plant_images/img12.jpg
img7.jpg
tractatus_plant_images/img7.jpg
img3.jpg
tractatus_plant_images/img3.jpg
img3.jpg
tractatus_plant_images/img3.jpg
img9.jpg
tractatus_plant_images/img9.jpg
img8.jpg
tractatus_plant_images/img8.jpg
img8.jpg
tractatus_plant_images/img8.jpg
img14.jpg
tractatus_plant_images/img14.jpg
img16.jpg
tractatus_plant_images/img16.jpg
img16.jpg
tractatus_plant_images/img16.jpg
img3.jpg
tractatus_plant_images/img3.jpg
img2.jpg
tractatus_plant_images/img2.jpg
img9.jpg
tractatus_plant_images/img9.jpg
img4.jpg
tractatus_plant_images/img4.jpg
img16.jpg
tractatus_plant_images/img16.jpg
img4.jpg
tractatus_plant_images/img4.jpg
img13.jpg
tractatus_plant_images/img13.jpg
img4.jpg
tractatus_plant_images/img4.jpg
img2.jpg
tractatus_plant_images/img2.jpg
img12.jpg
tractatus_plant_images/img12.jpg
img14.jpg
tractatus_plant_images/img14.jpg
img6.jpg
tractatus_plant_images/img6.jpg
img5.jpg
tractatus_plant_images/img5.jp

In [ ]:
!zip -r plant_images_flower_cleaned.zip plant_images_flower_cleaned/

  adding: plant_images_flower_cleaned/ (stored 0%)
  adding: plant_images_flower_cleaned/page7_1_cleaned.jpeg (deflated 59%)
  adding: plant_images_flower_cleaned/page119_30_cleaned.jpeg (deflated 72%)
  adding: plant_images_flower_cleaned/page40_20_cleaned.jpeg (deflated 96%)
  adding: plant_images_flower_cleaned/page72_9_cleaned.jpeg (deflated 95%)
  adding: plant_images_flower_cleaned/page24_19_cleaned.jpeg (deflated 92%)
  adding: plant_images_flower_cleaned/page170_3_cleaned.jpeg (deflated 95%)
  adding: plant_images_flower_cleaned/page119_32_cleaned.jpeg (deflated 90%)
  adding: plant_images_flower_cleaned/page19_6_cleaned.jpeg (deflated 93%)
  adding: plant_images_flower_cleaned/page81_2_cleaned.jpeg (deflated 28%)
  adding: plant_images_flower_cleaned/page65_3_cleaned.jpeg (deflated 89%)
  adding: plant_images_flower_cleaned/page24_22_cleaned.jpeg (deflated 94%)
  adding: plant_images_flower_cleaned/page43_8_cleaned.jpeg (deflated 88%)
  adding: plant_images_flower_cleaned/page